In [4]:
import tensorflow as tf

In [5]:
x = tf.constant([[1,2]])
neg_x = tf.negative(x)
print(neg_x)

Tensor("Neg_1:0", shape=(1, 2), dtype=int32)


In [7]:
with tf.Session() as sess:
    result = sess.run(neg_x)
    print(result)

[[-1 -2]]


In [11]:
##另一种关闭方式##

print(neg_x)
sess = tf.InteractiveSession()
neg_op = tf.negative(x)
result = neg_op.eval()

print(result)

sess.close

Tensor("Neg_1:0", shape=(1, 2), dtype=int32)
[[-1 -2]]


<bound method InteractiveSession.close of <tensorflow.python.client.session.InteractiveSession object at 0x0000000007CE8DD8>>

In [12]:
# 添加参数，配置
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    result = sess.run(neg_op)
    print(result)

[[-1 -2]]


In [17]:
## 使用变量，初始化变量 给变量赋值
tf.InteractiveSession()
raw_data = [1., 2., 8., -1., 0., 5.5, 6., 13]

# 必须初始化变量 
spike = tf.Variable(False)
# 下面这句可以注释掉
spike.initializer.run()

# assign(spike, tf.constant(True)) 将后面的值赋值给spike
for i in range(1, len(raw_data)):
    if raw_data[i] - raw_data[i-1] > 5:
        updater = tf.assign(spike, tf.constant(True))
        updater.eval()
    else:
        tf.assign(spike, False).eval()
    print("Spike", spike.eval())
    
sess.close()    

Spike False
Spike True
Spike False
Spike False
Spike True
Spike False
Spike True


In [ ]:
##保存变量
saver = tf.train.Saver()
save_path = saver.save(sess, "C:\\Users\\wilbur.wang\\AnacondaProjects\\TensorFlow-Book-master\\ch02_basics\\save")
print("spikes data saved in file: %s" % save_path)

saver = tf.train.Saver()

try:
    saver.restore(sess, 'spikes.ckpt')
    print(spikes.eval())
except:
    print('file not found')

In [ ]:
##线程和队列TensorFlow-Book-master/ch02_basics/Concept09_queue.ipynb
import multiprocessing
NUM_THREADS = multiprocessing.cpu_count()

xs = np.random.randn(100, 3)
ys = np.random.randint(0, 2, size=100)

## 定义队列和初始化
queue = tf.FIFOQueue(capacity=1000, dtypes=[tf.float32, tf.int32])

## 入列和出列
enqueue_op = queue.enqueue_many([xs, ys])
x_op, y_op = queue.dequeue()

## 开启四个线程http://www.jianshu.com/p/d063804fb272
qr = tf.train.QueueRunner(queue, [enqueue_op] * 4)

sess = tf.InteractiveSession()

coord = tf.train.Coordinator()
## 参看上面的连接 这个是启动队列线程 需要传sess，还有一个是全局线程
enqueue_threads = qr.create_threads(sess, coord=coord, start=True)

for _ in range(100):
    if coord.should_stop():
        break
    x, y = sess.run([x_op, y_op])
    print(x, y)
    
coord.request_stop()
coord.join(enqueue_threads)

In [ ]:
## read file http://norvig.com/big.txt

filename_queue = tf.train.string_input_producer(["big.txt"])
reader = tf.TextLineReader()
key_op, value_op = reader.read(filename_queue)

sess = tf.InteractiveSession()
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(coord=coord)

for i in range(100):
    key, value = sess.run([key_op, value_op])
    print(key, value)

In [ ]:
## Linear regression
learning_rate = 0.01
training_epochs = 100

x_train = np.linspace(-1, 1, 101)
y_train = 2 * x_train + np.random.randn(*x_train.shape) * 0.33

## y=wx
X = tf.placeholder("float")
Y = tf.placeholder("float")

w = tf.Variable(0.0, name="weights")

def model(X, w):
    return tf.multiply(X, w)

y_model = model(X, w)
cost = tf.reduce_mean(tf.square(Y-y_model))

train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

sess = tf.Session()

init = tf.global_variables_initializer()
sess.run(init)

for epoch in range(training_epochs):
    for (x, y) in zip(x_train, y_train):
        sess.run(train_op, feed_dict={X: x, Y: y})
## 经过学习 w_val = 1.91576    y_learned = x_train*w_val   
w_val = sess.run(w) 

sess.close()


In [ ]:
#Polynomial regression TensorFlow-Book-master/ch03_regression/Concept02_poly_regression.ipynb
for i in range(num_coeffs):
    trY += trY_coeffs[i] * np.power(trX, i)
# 添加噪音
trY += np.random.randn(*trX.shape) * 1.5

def model(X, w):
    terms = []
    for i in range(num_coeffs):
        term = tf.multiply(w[i], tf.pow(X, i))
        terms.append(term)
    return tf.add_n(terms)

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

for epoch in range(training_epochs):
    for (x, y) in zip(trX, trY):
        sess.run(train_op, feed_dict={X: x, Y: y})

w_val = sess.run(w)
print(w_val)
# 培训出来的是数组
[ 1.10158885  2.36433625  3.30378437  4.43473864  3.75751448  4.60356045]

In [1]:
## 伪造数据
x_dataset = np.linspace(-1, 1, 100)

num_coeffs = 9
y_dataset_params = [0.] * num_coeffs
y_dataset_params[2] = 1
y_dataset = 0
for i in range(num_coeffs):
    y_dataset += y_dataset_params[i] * np.power(x_dataset, i)
y_dataset += np.random.randn(*x_dataset.shape) * 0.3

print(y_dataset.shape)

## 将数据打乱 重新分割
def split_dataset(x_dataset, y_dataset, ratio):
    arr = np.arange(x_dataset.size)
    ## 打乱
    np.random.shuffle(arr)
    num_train = int(ratio * x_dataset.size)
    ##重新分配
    x_train = x_dataset[arr[0:num_train]]
    y_train = y_dataset[arr[0:num_train]]
    x_test = x_dataset[arr[num_train:x_dataset.size]]
    y_test = y_dataset[arr[num_train:x_dataset.size]]
    return x_train, x_test, y_train, y_test

(x_train, x_test, y_train, y_test) = split_dataset(x_dataset, y_dataset, 0.7)

w = tf.Variable([0.] * num_coeffs, name="parameters")
y_model = model(X, w)

## cost 函数，reg_lambda是程序变量 而不是tf变量
cost = tf.div(tf.add(tf.reduce_sum(tf.square(Y-y_model)),
                     tf.multiply(reg_lambda, tf.reduce_sum(tf.square(w)))),2*x_train.size)
train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

## reg_lambda 的使用，注意x_train 和 X: x_test
for reg_lambda in np.linspace(0,1,100):
    for epoch in range(training_epochs):
        sess.run(train_op, feed_dict={X: x_train, Y: y_train})
    final_cost = sess.run(cost, feed_dict={X: x_test, Y:y_test})


NameError: name 'np' is not defined

In [ ]:
## 评价  TensorFlow-Book-master/ch04_classification/Concept01_linear_regression_classification.ipynb
y_model = model(X, w)
cost = tf.reduce_sum(tf.square(Y-y_model))

correct_prediction = tf.equal(Y, tf.to_float(tf.greater(y_model, 0.5)))
accuracy = tf.reduce_mean(tf.to_float(correct_prediction))

w_val = sess.run(w)
print('learned parameters', w_val)
print('accuracy', sess.run(accuracy, feed_dict={X: xs, Y: labels}))

In [ ]:
## 逻辑回归 TensorFlow-Book-master/ch04_classification/Concept02_logistic.ipynb
X = tf.placeholder(tf.float32, shape=(None,), name="x")
Y = tf.placeholder(tf.float32, shape=(None,), name="y")
w = tf.Variable([0., 0.], name="parameter", trainable=True)
## 定义公式
y_model = tf.sigmoid(w[1] * X + w[0])
cost = tf.reduce_mean(-Y * tf.log(y_model) - (1 - Y) * tf.log(1 - y_model))

train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

## 注意 打印日志 y_model,cost，op操作类型是没有值得只是操作
 with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    prev_err = 0
    for epoch in range(training_epochs):
        y_model1,err, _ = sess.run([y_model,cost, train_op], {X: xs, Y: ys})
        if epoch % 100 == 0:
            print(epoch, err,y_model1[y_model1.size-1])
        if abs(prev_err - err) < 0.0001:
            break
        prev_err = err
    w_val = sess.run(w, {X: xs, Y: ys})


all_xs = np.linspace(-10, 10, 100)
with tf.Session() as sess:
    ## w_val[1] + w_val[0] 是上一个值 可以直接来用的
    predicted_vals = sess.run(tf.sigmoid(all_xs * w_val[1] + w_val[0]))
plt.plot(all_xs, predicted_vals)
plt.scatter(xs, ys)
plt.show()

In [ ]:
## http://blog.csdn.net/qingyang666/article/details/70285249?locationNum=6&fps=1  softmax和k元分类区别
## TensorFlow-Book-master/ch04_classification/Concept04_softmax.ipynb
### hstack 水平加列，横着加。 vstack垂直加 就是行
xs_label0 = np.hstack((x1_label0, x2_label0)) # x1_label0 (100, 1)
xs_label1 = np.hstack((x1_label1, x2_label1))
xs_label2 = np.hstack((x1_label2, x2_label2))

print(xs_label0.shape)#(100, 2)

xs = np.vstack((xs_label0, xs_label1, xs_label2))

print(xs.shape)#(300, 2)

labels = np.matrix([[1., 0., 0.]] * len(x1_label0) + [[0., 1., 0.]] * len(x1_label1) + [[0., 0., 1.]] * len(x1_label2))

print(labels.shape )#(300, 3)

test_xs = np.vstack((test_xs_label0, test_xs_label1, test_xs_label2))
test_labels = np.matrix([[1., 0., 0.]] * 10 + [[0., 1., 0.]] * 10 + [[0., 0., 1.]] * 10)
print([[1., 0., 0.]] * 2 +[[0., 1., 0.]] * 3)
print(test_labels.shape)
#[[1.0, 0.0, 0.0], [1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 1.0, 0.0], [0.0, 1.0, 0.0]]
#(30, 3)

train_size, num_features = xs.shape
print(xs.shape)#(300, 2)

X = tf.placeholder("float", shape=[None, num_features])
Y = tf.placeholder("float", shape=[None, num_labels])#num_labels = 3

W = tf.Variable(tf.zeros([num_features, num_labels]))#shape[2,3]
b = tf.Variable(tf.zeros([num_labels]))

y_model = tf.nn.softmax(tf.matmul(X, W) + b)

cost = -tf.reduce_sum(Y * tf.log(y_model))
train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

correct_prediction = tf.equal(tf.argmax(y_model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

with tf.Session() as sess:
    tf.global_variables_initializer().run()

    for step in range(training_epochs * train_size //batch_size):
        offset = (step * batch_size) % train_size
        batch_xs = xs[offset:(offset + batch_size), :]
        batch_labels = labels[offset:(offset + batch_size)]
        err, _ = sess.run([cost, train_op], feed_dict={X: batch_xs, Y: batch_labels})
        if step % 100 == 0:
            print (step, err)

    W_val = sess.run(W)
    print('w', W_val)
    b_val = sess.run(b)
    print('b', b_val)
    ## 最后两句写法一样
    print("accuracy", accuracy.eval(feed_dict={X: test_xs, Y: test_labels}))
    print("accuracy",sess.run(accuracy,feed_dict={X: test_xs, Y: test_labels}) )
    """
    w [[-2.47749233  0.20790951  2.26958179]
       [-0.43322     1.92567241 -1.49245799]]
    b [ 10.82922268  -2.62551188  -8.20370579]
    accuracy 0.966667
    accuracy 0.966667 
    """

In [ ]:
import tensorflow as tf
import numpy as np
x = np.asarray([[[1,2,3],[4,5,6]],
                [[7,8,9],[10,11,12]]])
x_p = tf.placeholder(tf.int32,[2,2,3])
y =  tf.reduce_sum(x_p,2) #修改这里
y1 =  tf.reduce_sum(x_p,1) #修改这里
y0 =  tf.reduce_sum(x_p,0) #修改这里
with tf.Session() as sess:
    y = sess.run(y,feed_dict={x_p:x})
    y1 = sess.run(y1,feed_dict={x_p:x})
    y0 = sess.run(y0,feed_dict={x_p:x})
    print ("y2: ",y)
    print ("y1: ",y1)
    print ("y0: ",y0)
"""
y2:  [[ 6 15]
      [24 33]]
y1:  [[ 5  7  9]
      [17 19 21]]
y0:  [[ 8 10 12]
      [14 16 18]]
"""    
## 聚类计算 TensorFlow-Book-master/ch05_clustering/Concept01_clustering.ipynb
def initial_cluster_centroids(X, k):
    return X[0:k, :]


def assign_cluster(X, centroids):
    expanded_vectors = tf.expand_dims(X, 0)
    expanded_centroids = tf.expand_dims(centroids, 1)
    distances = tf.reduce_sum(tf.square(tf.subtract(expanded_vectors, expanded_centroids)), 2)
    mins = tf.argmin(distances, 0)
    return mins


def recompute_centroids(X, Y):
    sums = tf.unsorted_segment_sum(X, Y, k)
    counts = tf.unsorted_segment_sum(tf.ones_like(X), Y, k)
    return sums / counts


In [4]:
import tensorflow as tf
import numpy as np

slice_location = [0, 0] ##[0,0]
slice_shape = [2, 1] ##[2,1]
obs_prob = np.array([[0.5, 0.4, 0.1], [0.1, 0.3, 0.6]])
initial_prob = np.array([[0.6], [0.4]])
with tf.Session() as sess:
    print(sess.run(tf.slice(obs_prob, slice_location, slice_shape)))
    fwd = tf.multiply(initial_prob, tf.slice(obs_prob, slice_location, slice_shape))
    print(sess.run(fwd))

[[ 0.5]
 [ 0.1]]
[[ 0.3 ]
 [ 0.04]]


In [ ]:
# HMM TensorFlow-Book-master/ch06_hmm/Concept01_forward.ipynb 使用了递归算法 反推的使用
# 还有一个类的使用 http://www.52nlp.cn/hmm-concrete-example-on-wiki

import numpy as np
import tensorflow as tf

class HMM(object):
    def __init__(self, initial_prob, trans_prob, obs_prob):
        self.N = np.size(initial_prob)
        self.initial_prob = initial_prob
        self.trans_prob = trans_prob
        self.emission = tf.constant(obs_prob)

        assert self.initial_prob.shape == (self.N, 1)
        assert self.trans_prob.shape == (self.N, self.N)
        assert obs_prob.shape[0] == self.N

        self.obs_idx = tf.placeholder(tf.int32)
        self.fwd = tf.placeholder(tf.float64)

    def get_emission(self, obs_idx):
        slice_location = [0, obs_idx] ##[0,0]
        num_rows = tf.shape(self.emission)[0] ##2
        slice_shape = [num_rows, 1] ##[2,1]
        return tf.slice(self.emission, slice_location, slice_shape)

    def forward_init_op(self):
        obs_prob = self.get_emission(self.obs_idx)#[[ 0.5] [ 0.1]]
        fwd = tf.multiply(self.initial_prob, obs_prob)
        return fwd

    def forward_op(self):
        transitions = tf.matmul(self.fwd, tf.transpose(self.get_emission(self.obs_idx)))
        weighted_transitions = transitions * self.trans_prob
        fwd = tf.reduce_sum(weighted_transitions, 0)
        return tf.reshape(fwd, tf.shape(self.fwd))
    
def forward_algorithm(sess, hmm, observations):
    
    fwd = sess.run(hmm.forward_init_op(), feed_dict={hmm.obs_idx: observations[0]})
    
    print(fwd)
    
    for t in range(1, len(observations)):
        fwd = sess.run(hmm.forward_op(), feed_dict={hmm.obs_idx: observations[t], hmm.fwd: fwd})
        print("fwd:",fwd)
    prob = sess.run(tf.reduce_sum(fwd))# [[ 0.00038915] [ 0.0002039 ]] 两个数相加
    return prob

if __name__ == '__main__':
    initial_prob = np.array([[0.6], [0.4]])
    trans_prob = np.array([[0.7, 0.3], [0.4, 0.6]])
    obs_prob = np.array([[0.5, 0.4, 0.1], [0.1, 0.3, 0.6]])

    hmm = HMM(initial_prob=initial_prob, trans_prob=trans_prob, obs_prob=obs_prob)
    
    # 因为obs_prob 有三个状态 所以索引是 01 2 注意参看连接文章
    observations = [0, 1, 1, 2, 1, 1, 1]
    with tf.Session() as sess:
        prob = forward_algorithm(sess, hmm, observations)
        print('Probability of observing {} is {}'.format(observations, prob))

In [ ]:
## TensorFlow-Book-master/ch07_autoencoder 无监督的分类
import tensorflow as tf
import numpy as np

def get_batch(X, size):
    a = np.random.choice(len(X), size, replace=False)
    return X[a]

class Autoencoder:
    def __init__(self, input_dim, hidden_dim, epoch=500, batch_size=10, learning_rate=0.001):
        self.epoch = epoch
        self.batch_size = batch_size
        self.learning_rate = learning_rate

        # Define input placeholder
        x = tf.placeholder(dtype=tf.float32, shape=[None, input_dim])
        
        # Define variables
        with tf.name_scope('encode'):
            weights = tf.Variable(tf.random_normal([input_dim, hidden_dim], dtype=tf.float32), name='weights')
            biases = tf.Variable(tf.zeros([hidden_dim]), name='biases')
            encoded = tf.nn.sigmoid(tf.matmul(x, weights) + biases)
        with tf.name_scope('decode'):
            weights = tf.Variable(tf.random_normal([hidden_dim, input_dim], dtype=tf.float32), name='weights')
            biases = tf.Variable(tf.zeros([input_dim]), name='biases')
            decoded = tf.matmul(encoded, weights) + biases

        self.x = x
        self.encoded = encoded
        self.decoded = decoded

        # Define cost function and training op
        self.loss = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(self.x, self.decoded))))

        self.all_loss = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(self.x, self.decoded)), 1))
        self.train_op = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss)
        
        # Define a saver op
        self.saver = tf.train.Saver()

    def train(self, data):
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for i in range(self.epoch):
                for j in range(500):
                    batch_data = get_batch(data, self.batch_size)
                    l, _ = sess.run([self.loss, self.train_op], feed_dict={self.x: batch_data})
                if i % 50 == 0:
                    print('epoch {0}: loss = {1}'.format(i, l))
                    self.saver.save(sess, './model.ckpt')
            self.saver.save(sess, './model.ckpt')
        
    def test(self, data):
        with tf.Session() as sess:
            self.saver.restore(sess, './model.ckpt')
            hidden, reconstructed = sess.run([self.encoded, self.decoded], feed_dict={self.x: data})
        print('input', data)
        print('compressed', hidden)
        print('reconstructed', reconstructed)
        return reconstructed

    def get_params(self):
        with tf.Session() as sess:
            self.saver.restore(sess, './model.ckpt')
            weights, biases = sess.run([self.weights1, self.biases1])
        return weights, biases

    def classify(self, data, labels):
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            self.saver.restore(sess, './model.ckpt')
            hidden, reconstructed = sess.run([self.encoded, self.decoded], feed_dict={self.x: data})
            reconstructed = reconstructed[0]
            # loss = sess.run(self.all_loss, feed_dict={self.x: data})
            print('data', np.shape(data))
            print('reconstructed', np.shape(reconstructed))
            loss = np.sqrt(np.mean(np.square(data - reconstructed), axis=1))
            print('loss', np.shape(loss))
            horse_indices = np.where(labels == 7)[0]
            not_horse_indices = np.where(labels != 7)[0]
            horse_loss = np.mean(loss[horse_indices])
            not_horse_loss = np.mean(loss[not_horse_indices])
            print('horse', horse_loss)
            print('not horse', not_horse_loss)
            return hidden[7,:]

    def decode(self, encoding):
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            self.saver.restore(sess, './model.ckpt')
            reconstructed = sess.run(self.decoded, feed_dict={self.encoded: encoding})
        img = np.reshape(reconstructed, (32, 32))
        return img
    
    
from sklearn import datasets

hidden_dim = 1
data = datasets.load_iris().data
input_dim = len(data[0])
ae = Autoencoder(input_dim, hidden_dim)
ae.train(data)
ae.test([[8, 4, 6, 2]])

"""input [[8, 4, 6, 2]]
compressed [[ 0.72223264]]
reconstructed [[ 6.87640762  2.79334426  6.23228502  2.21386957]]
"""

In [ ]:
## 自定义类 TensorFlow-Book-master/ch08_rl/Concept01_rl.ipynb
## TensorFlow-Book-master/ch09_cnn/Concept02_convolution CNN 查看单个图
# TensorFlow-Book-master/ch09_cnn/Concept03_cnn.ipynb
model_op = model()

cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=model_op, labels=y)
)
train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

correct_pred = tf.equal(tf.argmax(model_op, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [ ]:
# TensorFlow-Book-master/ch10_rnn/Concept02_rnn.ipynb RNN
#  tf.get_variable_scope().reuse_variables()的使用
# BasicLSTMCell 的使用  打开文件下载文件

import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn

class SeriesPredictor:

    def __init__(self, input_dim, seq_size, hidden_dim=10):# 1,4,10
        # Hyperparameters
        self.input_dim = input_dim
        self.seq_size = seq_size
        self.hidden_dim = hidden_dim

        # Weight variables and input placeholders
        self.W_out = tf.Variable(tf.random_normal([hidden_dim, 1]), name='W_out')#[10,1]
        self.b_out = tf.Variable(tf.random_normal([1]), name='b_out')
        self.x = tf.placeholder(tf.float32, [None, seq_size, input_dim])#[#,4,1]
        self.y = tf.placeholder(tf.float32, [None, seq_size])#[#,4]

        # Cost optimizer
        self.cost = tf.reduce_mean(tf.square(self.model() - self.y))
        self.train_op = tf.train.AdamOptimizer().minimize(self.cost)

        # Auxiliary ops
        self.saver = tf.train.Saver()

    def model(self):
        """
        :param x: inputs of size [T, batch_size, input_size]
        :param W: matrix of fully-connected output layer weights
        :param b: vector of fully-connected output layer biases
        """
        cell = rnn.BasicLSTMCell(self.hidden_dim)
        outputs, states = tf.nn.dynamic_rnn(cell, self.x, dtype=tf.float32)
        num_examples = tf.shape(self.x)[0]
        W_repeated = tf.tile(tf.expand_dims(self.W_out, 0), [num_examples, 1, 1])
        out = tf.matmul(outputs, W_repeated) + self.b_out
        out = tf.squeeze(out)
        return out

    def train(self, train_x, train_y):
        with tf.Session() as sess:
            tf.get_variable_scope().reuse_variables()
            sess.run(tf.global_variables_initializer())
            for i in range(1000):
                _, mse = sess.run([self.train_op, self.cost], feed_dict={self.x: train_x, self.y: train_y})
                if i % 100 == 0:
                    print(i, mse)
            save_path = self.saver.save(sess, 'model.ckpt')
            print('Model saved to {}'.format(save_path))

    def test(self, test_x):
        with tf.Session() as sess:
            tf.get_variable_scope().reuse_variables()
            self.saver.restore(sess, './model.ckpt')
            output = sess.run(self.model(), feed_dict={self.x: test_x})
            return output
        
        
if __name__ == '__main__':
    predictor = SeriesPredictor(input_dim=1, seq_size=4, hidden_dim=10)
    train_x = [[[1], [2], [5], [6]],
               [[5], [7], [7], [8]],
               [[3], [4], [5], [7]]]
    train_y = [[1, 3, 7, 11],
               [5, 12, 14, 15],
               [3, 7, 9, 12]]
    predictor.train(train_x, train_y)

    test_x = [[[1], [2], [3], [4]],  # 1, 3, 5, 7
              [[4], [5], [6], [7]]]  # 4, 9, 11, 13
    actual_y = [[[1], [3], [5], [7]],
                [[4], [9], [11], [13]]]
    pred_y = predictor.test(test_x)
    
    print("\nLets run some tests!\n")
    
    for i, x in enumerate(test_x):
        print("When the input is {}".format(x))
        print("The ground truth output should be {}".format(actual_y[i]))
        print("And the model thinks it is {}\n".format(pred_y[i])) ## 测试完的值 就是预测的输出
        
"""Lets run some tests!

When the input is [[1], [2], [3], [4]]
The ground truth output should be [[1], [3], [5], [7]]
And the model thinks it is [ 0.96018004  2.76944828  5.35826826  7.3706851 ]

When the input is [[4], [5], [6], [7]]
The ground truth output should be [[4], [9], [11], [13]]
And the model thinks it is [  4.17302942   9.161376    11.13204765  11.64120388]
"""

In [ ]:
## TensorFlow-Book-master/ch11_word2vec/Concept01_multi_rnn.ipynb
import tensorflow as tf
input_dim = 1
seq_size = 3
input_placeholder = tf.placeholder(dtype=tf.float32, shape=[None, seq_size, input_dim])

def make_cell(state_dim):
    return tf.contrib.rnn.LSTMCell(state_dim)

with tf.variable_scope("first_cell") as scope:
    cell = make_cell(state_dim=10)
    outputs, states = tf.nn.dynamic_rnn(cell, input_placeholder, dtype=tf.float32)

with tf.variable_scope("second_cell") as scope:
    cell2 = make_cell(state_dim=10)
    outputs2, states2 = tf.nn.dynamic_rnn(cell2, outputs, dtype=tf.float32)

def make_multi_cell(state_dim, num_layers):
    cells = [make_cell(state_dim) for _ in range(num_layers)]
    return tf.contrib.rnn.MultiRNNCell(cells)

multi_cell = make_multi_cell(state_dim=10, num_layers=5)
outputs5, states5 = tf.nn.dynamic_rnn(multi_cell, input_placeholder, dtype=tf.float32)

input_seq = [[1], [2], [3]]

init_op = tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(init_op)

outputs_val, outputs2_val, outputs5_val = sess.run([outputs, outputs2, outputs5], 
                                                   feed_dict={input_placeholder: [input_seq]})


    

In [ ]:
# TensorFlow-Book-master/ch11_word2vec/Concept02_embedding_lookup.ipynb
import tensorflow as tf
sess = tf.InteractiveSession()
embeddings_0d = tf.constant([17,22,35,51])

embeddings_4d = tf.constant([[1, 0, 0, 0],
                             [0, 1, 0, 0],
                             [0, 0, 1, 0],
                             [0, 0, 0, 1]])

embeddings_2x2d = tf.constant([[[1, 0], [0, 0]],
                               [[0, 1], [0, 0]],
                               [[0, 0], [1, 0]],
                               [[0, 0], [0, 1]]])

ids = tf.constant([1, 0, 2])

lookup_0d = sess.run(tf.nn.embedding_lookup(embeddings_0d, ids))
print(lookup_0d) #[22 17 35]

lookup_4d = sess.run(tf.nn.embedding_lookup(embeddings_4d, ids))
print(lookup_4d)

"""
[[0 1 0 0]
 [1 0 0 0]
 [0 0 1 0]]
"""
lookup_2x2d = sess.run(tf.nn.embedding_lookup(embeddings_2x2d, ids))
print(lookup_2x2d)

"""
[[[0 1]
  [0 0]]

 [[1 0]
  [0 0]]

 [[0 0]
  [1 0]]]
"""

In [ ]:
##  https://hackernoon.com/deep-learning-cnns-in-tensorflow-with-gpus-cba6efe0acc2 完整的例子，评估 恢复数据  类

# 归一化
def normalize(x):
   
    normalized_x = ( x - x.min())  /( x.max() - x.min() )
    return normalized_x

#tf.train.import_meta_graph  参看下面的连接另一种保存 
#另一种保存 还原方式 https://github.com/vyomshm/Image-Classification-Project/blob/master/dlnd_image_classification.ipynb 比较商业的代码

In [ ]:
# https://www.bonaccorso.eu/2017/08/02/svd-recommendations-using-tensorflow/  推荐系统归类
# https://blog.altoros.com/using-k-means-clustering-in-tensorflow.html 非监督学习 无须标签 K聚类
# https://www.altoros.com/performance-benchmark-distributed-tensorflow.html 分布式多核配置

In [ ]:
# tf.slice http://blog.csdn.net/chenxieyy/article/details/53031943  
import tensorflow as tf
import numpy as np
x=[[1,2,3],[4,5,6]]
y=np.arange(24).reshape([2,3,4])
z=tf.constant([[[1,2,3],[4,5,6]], [[7,8,9],[10,11,12]],  [[13,14,15],[16,17,18]]]
sess=tf.Session()
begin_x=[1,0]        #第一个1，决定了从x的第二行[4,5,6]开始，第二个0，决定了从[4,5,6] 中的4开始抽取
size_x=[1,2]           # 第一个1决定了，从第二行以起始位置抽取1行，也就是只抽取[4,5,6] 这一行，在这一行中从4开始抽取2个元素
out=tf.slice(x,begin_x,size_x)
print sess.run(out)  #  结果:[[4 5]]

begin_y=[1,0,0]
size_y=[1,2,3]
out=tf.slice(y,begin_y,size_y)   
print sess.run(out)  # 结果:[[[12 13 14] [16 17 18]]]

begin_z=[0,1,1]
size_z=[-1,1,2] 
out=tf.slice(z,begin_z,size_z)
print sess.run(out)  # size[i]=-1 表示第i维从begin[i]剩余的元素都要被抽取，结果：[[[ 5  6]] [[11 12]] [[17 18]]]


In [ ]:
# tf.gather   https://blog.altoros.com/using-k-means-clustering-in-tensorflow.html
import tensorflow as tf

temp = tf.range(0,10)*10 + tf.constant(1,shape=[10])
temp2 = tf.gather(temp,[1,5,9])

with tf.Session() as sess:

    print sess.run(temp)
    print sess.run(temp2)

"""
[ 1 11 21 31 41 51 61 71 81 91] 
[11 51 91]
"""    

In [ ]:
"""
Below is an overview of the 5 steps in the LSTM model life-cycle in Keras that we are going to look at.

Define Network
Compile Network
Fit Network
Evaluate Network
Make Predictions
"""

In [ ]:
## 下面链接是conda 环境配置 更新，版本查看
## http://webcache.googleusercontent.com/search?q=cache:http://machinelearningmastery.com/setup-python-environment-machine-learning-deep-learning-anaconda/

## 销量预测 讲解了 划分数据  数据归一化
## LSTM http://machinelearningmastery.com/multi-step-time-series-forecasting-long-short-term-memory-networks-python/

## LSTM 数据准备  天气预测
## http://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/

"""http://localhost:8888/notebooks/AnacondaProjects/TensorFlowStudy.ipynb#
http://machinelearningmastery.com/custom-metrics-deep-learning-keras-python/
Keras Classification Metrics

Below is a list of the metrics that you can use in Keras on classification problems.

Binary Accuracy: binary_accuracy, acc
Categorical Accuracy: categorical_accuracy, acc
Sparse Categorical Accuracy: sparse_categorical_accuracy
Top k Categorical Accuracy: top_k_categorical_accuracy (requires you specify a k parameter)
Sparse Top k Categorical Accuracy: sparse_top_k_categorical_accuracy (requires you specify a k parameter)
Accuracy is special.

Regardless of whether your problem is a binary or multi-class classification problem, you can specify the ‘acc‘ metric to report on accuracy.
"""

# 线性回归 loss 函数  使用单独的公式

# L1-Loss
--------------

Here, we will illustrate linear regression with the L1-Loss. Later in this script, we will illustrate the same problem with L2-Loss.

The equation for the L1 Loss for Linear Least Squares is

$$S = \sum_{i=1}^{N}\left| y_{i} - \hat{y_{i}} \right|$$

Where $N$ is the number of data points, $y_{i}$ is the i-th actual y-values, $\hat{y_{i}}$ is the predicted i-th y-value.

We start a computational graph session.

# L2-Loss
--------

Here, we will illustrate linear regression with the L2-Loss..

The equation for the L2 Loss for Linear Least Squares is

$$S = \sum_{i=1}^{N}\left( y_{i} - \hat{y_{i}} \right)^{2}$$

Where $N$ is the number of data points, $y_{i}$ is the i-th actual y-values, $\hat{y_{i}}$ is the predicted i-th y-value.

We start a computational graph session.

In [ ]:
#tensorflow_cookbook-master/03_Linear_Regression/04_Loss_Functions_in_Linear_Regressions/04_lin_reg_l1_vs_l2.ipynb
# Declare model operations
model_output = tf.add(tf.matmul(x_data, A), b)
# Declare loss functions
loss_l1 = tf.reduce_mean(tf.abs(y_target - model_output))

# Declare optimizers
my_opt_l1 = tf.train.GradientDescentOptimizer(learning_rate)
train_step_l1 = my_opt_l1.minimize(loss_l1)

# Initialize variables
init = tf.global_variables_initializer()
sess.run(init)

#______________________________________________

# Declare model operations
model_output = tf.add(tf.matmul(x_data, A), b)
# Declare loss functions
loss_l2 = tf.reduce_mean(tf.square(y_target - model_output))

# Declare optimizers
my_opt_l2 = tf.train.GradientDescentOptimizer(learning_rate)
train_step_l2 = my_opt_l2.minimize(loss_l2)

# Initialize variables
init = tf.global_variables_initializer()
sess.run(init)



# Deming Regression
-------------------------------

This function shows how to use TensorFlow to solve linear Deming regression.

$y = Ax + b$

We will use the iris data, specifically:

y = Sepal Length and x = Petal Width.

Demming regression is also called total least squares, in which we minimize the shortest distance from the predicted line and the actual (x,y) points.

If least squares linear regression minimizes the vertical distance to the line, Deming regression minimizes the total distance to the line.  This type of regression minimizes the error in the y values and the x values.  See the below figure for a comparison.

<img src="../images/05_demming_vs_linear_reg.png" width="512">

To implement this in TensorFlow, we start by loading the necessary libraries.

For the demming loss, we want to compute:

$$ \frac{\left| A \cdot x + b - y \right|}{\sqrt{A^{2} + 1}} $$

Which will give us the shortest distance between a point (x,y) and the predicted line, $A \cdot x + b$.

In [ ]:
# tensorflow_cookbook-master/03_Linear_Regression/05_Implementing_Deming_Regression/05_demming_regression.ipynb
# Declare model operations
model_output = tf.add(tf.matmul(x_data, A), b)
# Declare Demming loss function
demming_numerator = tf.abs(tf.subtract(tf.add(tf.matmul(x_data, A), b), y_target))
demming_denominator = tf.sqrt(tf.add(tf.square(A),1))
loss = tf.reduce_mean(tf.truediv(demming_numerator, demming_denominator))

# Declare optimizer
my_opt = tf.train.GradientDescentOptimizer(0.25)
train_step = my_opt.minimize(loss)

# Initialize variables
init = tf.global_variables_initializer()
sess.run(init)

In [1]:
# tensorflow_cookbook-master/03_Linear_Regression/06_Implementing_Lasso_and_Ridge_Regression/06_lasso_and_ridge_regression.ipynb
# Declare model operations
model_output = tf.add(tf.matmul(x_data, A), b)
# Select appropriate loss function based on regression type

if regression_type == 'LASSO':
    # Declare Lasso loss function
    # Lasso Loss = L2_Loss + heavyside_step,
    # Where heavyside_step ~ 0 if A < constant, otherwise ~ 99
    lasso_param = tf.constant(0.9)
    heavyside_step = tf.truediv(1., tf.add(1., tf.exp(tf.multiply(-50., tf.subtract(A, lasso_param)))))
    regularization_param = tf.multiply(heavyside_step, 99.)
    loss = tf.add(tf.reduce_mean(tf.square(y_target - model_output)), regularization_param)

elif regression_type == 'Ridge':
    # Declare the Ridge loss function
    # Ridge loss = L2_loss + L2 norm of slope
    ridge_param = tf.constant(1.)
    ridge_loss = tf.reduce_mean(tf.square(A))
    loss = tf.expand_dims(tf.add(tf.reduce_mean(tf.square(y_target - model_output)), tf.multiply(ridge_param, ridge_loss)), 0)
    
else:
    print('Invalid regression_type parameter value',file=sys.stderr)
    
# Declare optimizer
my_opt = tf.train.GradientDescentOptimizer(0.001)
train_step = my_opt.minimize(loss)

NameError: name 'tf' is not defined

In [ ]:
#tensorflow_cookbook-master/03_Linear_Regression/07_Implementing_Elasticnet_Regression/07_elasticnet_regression.ipynb
# Create variables for linear regression
A = tf.Variable(tf.random_normal(shape=[3,1]))
b = tf.Variable(tf.random_normal(shape=[1,1]))

# Declare model operations
model_output = tf.add(tf.matmul(x_data, A), b)

# Declare the elastic net loss function
elastic_param1 = tf.constant(1.)
elastic_param2 = tf.constant(1.)
l1_a_loss = tf.reduce_mean(tf.abs(A))
l2_a_loss = tf.reduce_mean(tf.square(A))
e1_term = tf.multiply(elastic_param1, l1_a_loss)
e2_term = tf.multiply(elastic_param2, l2_a_loss)
loss = tf.expand_dims(tf.add(tf.add(tf.reduce_mean(tf.square(y_target - model_output)), e1_term), e2_term), 0)

# Declare optimizer
my_opt = tf.train.GradientDescentOptimizer(0.001)
train_step = my_opt.minimize(loss)

In [ ]:
#tensorflow_cookbook-master/03_Linear_Regression/08_Implementing_Logistic_Regression/08_logistic_regression.ipynb
# 逻辑回归  数据处理的知识，下载解析

# Declare model operations
model_output = tf.add(tf.matmul(x_data, A), b)

# Declare loss function (Cross Entropy loss)
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=model_output, labels=y_target))

# Declare optimizer
my_opt = tf.train.GradientDescentOptimizer(0.01)
train_step = my_opt.minimize(loss)

# Actual Prediction 预测的方法 统计
prediction = tf.round(tf.sigmoid(model_output))
predictions_correct = tf.cast(tf.equal(prediction, y_target), tf.float32)
accuracy = tf.reduce_mean(predictions_correct)



# Logistic Regression

This function shows how to use TensorFlow to solve logistic regression.
$ \textbf{y} = sigmoid(\textbf{A}\times \textbf{x} + \textbf{b})$

In [ ]:
#/tensorflow_cookbook-master/04_Support_Vector_Machines/02_Working_with_Linear_SVMs/02_linear_svm.ipynb
# 所有的tensorflow_cookbook-master 的readme 都有图做说明，这个打不开 请在github中打开查看 很有用
# Declare model operations
model_output = tf.subtract(tf.matmul(x_data, A), b)

# Declare vector L2 'norm' function squared
l2_norm = tf.reduce_sum(tf.square(A))

# Declare loss function
# Loss = max(0, 1-pred*actual) + alpha * L2_norm(A)^2
# L2 regularization parameter, alpha
alpha = tf.constant([0.01])
# Margin term in loss
classification_term = tf.reduce_mean(tf.maximum(0., tf.subtract(1., tf.multiply(model_output, y_target))))
# Put terms together
loss = tf.add(classification_term, tf.multiply(alpha, l2_norm))

# Declare prediction function
prediction = tf.sign(model_output)
accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, y_target), tf.float32))

# Declare optimizer
my_opt = tf.train.GradientDescentOptimizer(0.01)
train_step = my_opt.minimize(loss)

# Initialize variables
init = tf.global_variables_initializer()
sess.run(init)

Declare our model and L2 Norm

SVM linear model is given by the equation:

$$\left[ \frac{1}{n} \sum_{i=1}^{n} \max(0, 1 - A \cdot x - b) \right] + \alpha \cdot ||A||^{2}$$

Our loss function will be the above quantity and we will tell TensorFlow to minimize it. Note that $n$ is the number of points (in a batch), $A$ is the hyperplane-normal vector (to solve for), $b$ is the hyperplane-offset (to solve for), and $\alpha$ is the soft-margin parameter.

In [ ]:
# tensorflow_cookbook-master/04_Support_Vector_Machines/04_Working_with_Kernels/04_svm_kernels.ipynb 不是线性的 可以参看图，公式比较复杂
# Apply kernel
# Linear Kernel
# my_kernel = tf.matmul(x_data, tf.transpose(x_data))

# Gaussian (RBF) kernel
gamma = tf.constant(-50.0)
dist = tf.reduce_sum(tf.square(x_data), 1)
dist = tf.reshape(dist, [-1,1])
sq_dists = tf.add(tf.subtract(dist, tf.multiply(2., tf.matmul(x_data, tf.transpose(x_data)))), tf.transpose(dist))
my_kernel = tf.exp(tf.multiply(gamma, tf.abs(sq_dists)))

# Compute SVM Model
first_term = tf.reduce_sum(b)
b_vec_cross = tf.matmul(tf.transpose(b), b)
y_target_cross = tf.matmul(y_target, tf.transpose(y_target))
second_term = tf.reduce_sum(tf.multiply(my_kernel, tf.multiply(b_vec_cross, y_target_cross)))
loss = tf.negative(tf.subtract(first_term, second_term))

# Create Prediction Kernel
# Linear prediction kernel
# my_kernel = tf.matmul(x_data, tf.transpose(prediction_grid))

# Gaussian (RBF) prediction kernel
rA = tf.reshape(tf.reduce_sum(tf.square(x_data), 1),[-1,1])
rB = tf.reshape(tf.reduce_sum(tf.square(prediction_grid), 1),[-1,1])
pred_sq_dist = tf.add(tf.subtract(rA, tf.multiply(2., tf.matmul(x_data, tf.transpose(prediction_grid)))), tf.transpose(rB))
pred_kernel = tf.exp(tf.multiply(gamma, tf.abs(pred_sq_dist)))

prediction_output = tf.matmul(tf.multiply(tf.transpose(y_target),b), pred_kernel)
prediction = tf.sign(prediction_output-tf.reduce_mean(prediction_output))
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.squeeze(prediction), tf.squeeze(y_target)), tf.float32))

SVM 非线性 多元分类都支持
tensorflow_cookbook-master/04_Support_Vector_Machines/06_Implementing_Multiclass_SVMs/06_multiclass_svm.ipynb
tensorflow_cookbook-master/04_Support_Vector_Machines/05_Implementing_Nonlinear_SVMs/05_nonlinear_svm.ipynb

### Gaussian (RBF) Kernel

We create the gaussian kernel that is used to transform the data points into a higher dimensional space.

The Kernel of two points, $x$ and $x'$ is given as

$$K(x, x')=exp\left(-\gamma|| x-x' ||^{2}\right)$$

For $\gamma$ very small, the kernel is very wide, and vice-versa for large $\gamma$ values.  This means that large $\gamma$ leads to high bias and low variance models.

If we have a vector of points, $x$ of size (batch_size, 2), then our kernel calculation becomes

$$K(\textbf{x})=exp\left( -\gamma \textbf{x} \cdot \textbf{x}^{T} \right)$$

### Compute SVM Model

Here, the SVM loss is given by two terms, The first term is the sum of the $b$ matrix, and the second term is 

$$\sum\left(K\cdot||\textbf{b}||^{2}||\textbf{y}||^{2}\right)$$

We finally tell TensorFlow to maximize the loss by minimizing the negative:  (The following is a horribly abbreviated version of the dual problem)

$$-\left(\sum\textbf{b} - \sum\left(K\cdot||\textbf{b}||^{2}||\textbf{y}||^{2}\right)\right)$$

In [ ]:
# Gaussian (RBF) kernel
gamma = tf.constant(-50.0)
sq_vec = tf.multiply(2., tf.matmul(x_data, tf.transpose(x_data)))
my_kernel = tf.exp(tf.multiply(gamma, tf.abs(sq_vec)))

# Compute SVM Model
first_term = tf.reduce_sum(b)
b_vec_cross = tf.matmul(tf.transpose(b), b)
y_target_cross = tf.matmul(y_target, tf.transpose(y_target))
second_term = tf.reduce_sum(tf.multiply(my_kernel, tf.multiply(b_vec_cross, y_target_cross)))
loss = tf.negative(tf.subtract(first_term, second_term))

# Gaussian (RBF) prediction kernel
rA = tf.reshape(tf.reduce_sum(tf.square(x_data), 1),[-1,1])
rB = tf.reshape(tf.reduce_sum(tf.square(prediction_grid), 1),[-1,1])
pred_sq_dist = tf.add(tf.subtract(rA, tf.multiply(2., tf.matmul(x_data, tf.transpose(prediction_grid)))), tf.transpose(rB))
pred_kernel = tf.exp(tf.multiply(gamma, tf.abs(pred_sq_dist)))

prediction_output = tf.matmul(tf.multiply(tf.transpose(y_target),b), pred_kernel)
prediction = tf.sign(prediction_output-tf.reduce_mean(prediction_output))
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.squeeze(prediction), tf.squeeze(y_target)), tf.float32))

# Declare optimizer
my_opt = tf.train.GradientDescentOptimizer(0.01)
train_step = my_opt.minimize(loss)

# Initialize variables
init = tf.global_variables_initializer()
sess.run(init)